# **Lab 1: Matrix algorithms**
**Kristoffer Almroth**

# **Abstract**

First lab in the course DD2363 Methods in Scientific Computing. This lab is about basic matrix operations.

# **Set up environment**

Dependencies needed for running the code.

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D
from random import randrange

import unittest

# **Introduction**

The aim with this lab is to get familiar with the basic matrix operations and the library numpy in python. The following operations were implemented: dot product, scalar-matrix multiplication, matrix-matrix multiplication, euclidian norm and euclidian distance.



# **Methods**

This section is based on the theory from the first two lectures. All code is tested using various different test cases, including a test case with a large number of randomly generated matrices and vectors. The result from the random test cases is then compared with the corresponding functions in the library numpy, which has been rigorously tested and can be seen as accurate. Since floating point precision were used in the tests, there could be rounding errors. np.testing.assert_allclose is used to compare matrices up to a specific precision, here 1e-5. All functions also include a dimension check to verify that the input is valid. 

Scalar product, also called dot product, can be seen as the sum of elementwise multiplication of two vectors.

In [0]:
def scalarProduct(v1, v2):

  # Check dimensions
  assert v1.shape == v2.shape

  res = 0
  for i in range(0, v1.size):
    res += v1[i]*v2[i]
  return res

class Test(unittest.TestCase):

  def testWrongDimensions(self):
    v1 = np.array([1,2,3,4])
    v2 = np.array([1,2,3,4,5])
    with self.assertRaises(AssertionError):
      scalarProduct(v1, v2)

  def testZeroVector(self):
    v1 = np.array([1,2,3,4])
    v2 = np.array([0,0,0,0])
    self.assertEquals(scalarProduct(v1, v2), 0)

  def testRandom(self):
    for i in range(0,1000):
      size = randrange(100) + 1
      v1 = np.random.rand(size)
      v2 = np.random.rand(size)
      np.testing.assert_allclose(scalarProduct(v1, v2), np.dot(v1,v2), rtol=1e-5, atol=0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


../usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Please use assertEqual instead.
.
----------------------------------------------------------------------
Ran 3 tests in 0.144s

OK


Multiplicating a matrix with a vector follows the normal rules of matrix multiplication. In numpy, multiplying a matrix with a vector using normal matrix multiplication when both are of the datatype np.array will create a (n\*n) matrix and not a (n\*1) as expected, so matrix.dot(vector) is used instead. Could also be solved by specifying the type of the matrix to be np.matrix.

In [0]:
def matrixVectorProduct(v, m):

  mCols = m.shape[1]
  mRows = m.shape[0]

  # Check dimensions
  assert v.size == mCols

  res = np.zeros((mRows))
  for i in range(0, mRows):
    for j in range(0, mCols):
      res[i] += m[i,j]*v[j]
  return res

class Test(unittest.TestCase):

  def testWrongDimensions(self):
    v = np.array([1,2,3])
    m = np.array([[2,4,6,1], [3,6,9,0], [1,2,1,3]])
    with self.assertRaises(AssertionError):
      matrixVectorProduct(v, m)

  def testZeroVector(self):
    v = np.array([0,0,0,0])
    m = np.array([[2,4,6,1], [3,6,9,0], [1,2,1,3]])
    res = np.array([0,0,0])
    self.assertTrue(np.array_equal(matrixVectorProduct(v, m), res))

  def testRandom(self):
    for i in range(0,1000):
      row = randrange(100) + 1
      col = randrange(100) + 1
      m = np.random.rand(row,col)
      v = np.random.rand(col)
      np.testing.assert_allclose(matrixVectorProduct(v, m), m.dot(v), rtol=1e-5, atol=0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


...
----------------------------------------------------------------------
Ran 3 tests in 2.188s

OK


In matrix multiplication the shape of the two matrices must be in the form (n\*l) and (l\*m) for it to work. The simple algorithm from the first lecture is used. 

In [0]:
def matrixMatrixProduct(m1, m2):

  m1Cols = m1.shape[1]
  m1Rows = m1.shape[0]
  m2Cols = m2.shape[1]
  m2Rows = m2.shape[0]

  # Check dimensions
  assert m1Cols == m2Rows

  res = np.zeros(shape=(m1Rows,m2Cols))
  for i in range(0, m1Rows):
    for j in range(0, m2Cols):
      value = 0
      for k in range(0, m1Cols):
        value += m1[i, k] * m2[k, j]
      res[i, j] = value
  return res

class Test(unittest.TestCase):

  def testWrongDimensions(self):
    m = np.array([[2,4,6,1],[3,6,9,0],[1,2,1,3]])
    with self.assertRaises(AssertionError):
      matrixMatrixProduct(m, m)

  def testZeroVector(self):
    m1 = np.array([[2,4,6,1],[3,6,9,0],[1,2,1,3]])
    m2 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    m0 = np.array([[0,0,0],[0,0,0],[0,0,0]])
    self.assertTrue(np.array_equal(matrixMatrixProduct(m1, m2), m0))

  def testRandom(self):
    for i in range(0,100):
      row = randrange(100) + 1
      col = randrange(100) + 1
      col2 = randrange(100) + 1
      m1 = np.random.rand(row,col)
      m2 = np.random.rand(col,col2)
      np.testing.assert_allclose(matrixMatrixProduct(m1, m2), np.matmul(m1,m2), rtol=1e-5, atol=0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


...
----------------------------------------------------------------------
Ran 3 tests in 7.850s

OK


The norm is calculated by taking the square root of the sum of elementwise multiplication of two vectors. This can be simplified by taking the square root of the dot product of a vector with itself, as shown in the first lecture. 

In [0]:
def euclidianNorm(v):
  return np.sqrt(scalarProduct(v,v))

class Test(unittest.TestCase):

  def testZeroVector(self):
    v = np.array([0,0,0,0])
    self.assertEquals(euclidianNorm(v), 0)

  def testRandom(self):
    for i in range(0,10000):
      row = randrange(100) + 1
      v = np.random.rand(row)
      np.testing.assert_allclose(euclidianNorm(v), np.linalg.norm(v), rtol=1e-5, atol=0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


./usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Please use assertEqual instead.
  
.
----------------------------------------------------------------------
Ran 2 tests in 1.292s

OK


Calculating the euclidean distance between two points is the same as taking the norm of the first vector subtracted with the second one. This means that two functions needed to be created, one for calculating the subtraction between two vectors and one for the euclidean distance. Both of these functions are tested.

In [0]:
def vectorSubtraction(v1, v2):

  # Check dimensions
  assert v1.shape == v2.shape

  res = np.zeros((v1.size))
  for i in range(0, v1.size):
    res[i] = v1[i]-v2[i]
  return res

def metric(v1, v2):
  return euclidianNorm(vectorSubtraction(v1,v2))

class Test(unittest.TestCase):

  def testVectorSubtractionWrongDimensions(self):
    v1 = np.array([1,2,3,4])
    v2 = np.array([1,2,3,4,5])
    with self.assertRaises(AssertionError):
      vectorSubtraction(v1, v2)

  def testMetricWrongDimensions(self):
    v1 = np.array([1,2,3,4])
    v2 = np.array([1,2,3,4,5])
    with self.assertRaises(AssertionError):
      metric(v1, v2)

  def testVectorSubtractionZeroVector(self):
    v1 = np.array([1,2,3,4])
    v2 = np.array([0,0,0,0])
    self.assertTrue(np.array_equal(vectorSubtraction(v1,v2), v1))

  def testMetricZeroVector(self):
    v1 = np.array([1,2,3,4])
    v2 = np.array([0,0,0,0])
    self.assertTrue(np.array_equal(metric(v1,v2), euclidianNorm(v1)))

  def testRandomVectorSubtraction(self):
    for i in range(0,1000):
      row = randrange(100) + 1
      v1 = np.random.rand(row)
      v2 = np.random.rand(row)
      np.testing.assert_allclose(vectorSubtraction(v1, v2), np.subtract(v1,v2), rtol=1e-5, atol=0)

  def testRandomMetric(self):
    for i in range(0,1000):
      row = randrange(100) + 1
      v1 = np.random.rand(row)
      v2 = np.random.rand(row)
      np.testing.assert_allclose(metric(v1, v2), np.linalg.norm(v1-v2), rtol=1e-5, atol=0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


......
----------------------------------------------------------------------
Ran 6 tests in 0.319s

OK


# **Results**

The lab was about creating functions for some simple matrix and vector operations. Below is all functions created, without the test code. For more information about the functions, see Methods. 

In [0]:
def scalarProduct(v1, v2):

  # Check dimensions
  assert v1.shape == v2.shape

  res = 0
  for i in range(0, v1.size):
    res += v1[i]*v2[i]
  return res

def matrixVectorProduct(v, m):

  mCols = m.shape[1]
  mRows = m.shape[0]

  # Check dimensions
  assert v.size == mCols

  res = np.zeros((mRows))
  for i in range(0, mRows):
    for j in range(0, mCols):
      res[i] += m[i,j]*v[j]
  return res

def matrixMatrixProduct(m1, m2):

  m1Cols = m1.shape[1]
  m1Rows = m1.shape[0]
  m2Cols = m2.shape[1]
  m2Rows = m2.shape[0]

  # Check dimensions
  assert m1Cols == m2Rows

  res = np.zeros(shape=(m1Rows,m2Cols))
  for i in range(0, m1Rows):
    for j in range(0, m2Cols):
      value = 0
      for k in range(0, m1Cols):
        value += m1[i, k] * m2[k, j]
      res[i, j] = value
  return res

def euclidianNorm(v):
  return np.sqrt(scalarProduct(v,v))

def vectorSubtraction(v1, v2):

  # Check dimensions
  assert v1.shape == v2.shape

  res = np.zeros((v1.size))
  for i in range(0, v1.size):
    res[i] = v1[i]-v2[i]
  return res

def metric(v1, v2):
  return euclidianNorm(vectorSubtraction(v1,v2))

# **Discussion**

Simple matrix functions is something that I have already done in previous courses, for example DD1388 Programsystemkonstruktion med C++. Thus there were no real challenge in implementing these functions except for the labs being done in python, which I am not that familiar with. Python is not that hard to learn so it should not take that much time before I am comfortable with its syntax. 